# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Функция для вывода таблиц:

In [2]:
def result(query):
    display(pd.io.sql.read_sql(query, con = engine))

Таблица books

In [3]:
books = ''' SELECT * 
            FROM books
            LIMIT 5
        '''
result(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors

In [4]:
authors = ''' SELECT * 
              FROM authors
              LIMIT 5
          '''
result(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица publishers

In [5]:
publishers = ''' SELECT * 
                 FROM publishers
                 LIMIT 5
             '''
result(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица ratings

In [6]:
ratings = ''' SELECT * 
              FROM ratings
              LIMIT 5
          '''
result(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews

In [7]:
reviews = ''' SELECT * 
              FROM reviews
              LIMIT 5
          '''
result(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Задача:** Cколько книг вышло после 1 января 2000 года?

In [8]:
data_1 = ''' SELECT COUNT(book_id)
             FROM books
             WHERE publication_date > '2000-01-01' 
         '''
result(data_1)

,count
0,819


**Ответ:** После 1 января 2000 года вышло 819 книг.

**Задача:** Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [9]:
data_2 = ''' SELECT b.title, b.book_id, count (DISTINCT review_id) AS review, avg (rating) AS rating 
             FROM books AS b
             LEFT JOIN reviews ON reviews.book_id = b.book_id
             LEFT JOIN ratings ON ratings.book_id  = b.book_id
             GROUP BY b.book_id
             ORDER BY COUNT(reviews) DESC
          '''
result(data_2)

,title,book_id,review,rating
0,Twilight (Twilight #1),948,7,3.662500
1,The Hobbit or There and Back Again,750,6,4.125000
2,The Catcher in the Rye,673,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.287500
...,...,...,...,...
995,Essential Tales and Poems,221,0,4.000000
996,Leonardo's Notebooks,387,0,4.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667


**Ответ:** Созданы соответствующие столбцы.

**Задача:** Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [10]:
data_3 = ''' SELECT p.publisher, COUNT(b.book_id) AS number_of_books
             FROM books AS b
             LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
             WHERE b.num_pages > 50
             GROUP BY p.publisher
             ORDER BY number_of_books DESC
             LIMIT 1
         '''
result(data_3)

,publisher,number_of_books
0,Penguin Books,42


**Ответ:** Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц - 42 штуки.

**Задача:** Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [11]:
data_4 = ''' SELECT a.author, AVG(r.rating) AS mean_rating
             FROM (SELECT book_id, AVG(rating) AS rating_avg
                   FROM ratings
                   GROUP BY book_id
                   HAVING COUNT(rating_id) >= 50) AS rt
             JOIN ratings AS r on rt.book_id=r.book_id 
             JOIN books AS b ON rt.book_id = b.book_id
             JOIN authors AS a ON b.author_id = a.author_id
             GROUP BY a.author       
             ORDER BY mean_rating DESC
             LIMIT 1
         '''
result(data_4)

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Ответ:** Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг книг - 4.28.

**Задача:** Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
data_5 = ''' SELECT 
             COUNT(review_id)/ COUNT(DISTINCT(username)) AS mean_review 
             FROM reviews
             WHERE  username IN 
                 (SELECT username
                  FROM ratings
                  GROUP BY username
                  HAVING COUNT(rating_id) > 50)
         '''
result(data_5)

,mean_review
0,24


**Ответ:** Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 штуки.

**ВЫВОДЫ:**

- После 1 января 2000 года вышло 819 книг.
- Для каждой книги выведены количество обзоров и средняя оценка.
- Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц - 42 штуки.
- Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг книг - 4.28.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 штуки.